In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

is_local = False

function_name = "extracttomarkdown"

if is_local:
    url = "http://localhost:7071/api/"
    
else:
    url = "https://genai-extact-from-pdf.azurewebsites.net/api/"
    
    
url = url + function_name
    




In [ ]:
import os

# Test 1        
pdf_url1 = os.getenv('PDF_URL_1')
pdf_sas1 = os.getenv('PDF_SAS_1')

pdf_url2 = os.getenv('PDF_URL_2')
pdf_sas2 = os.getenv('PDF_SAS_2')

In [ ]:
import requests
import json



data = {
    "values": [
        {
            "recordId": "record1",
            "data": { 
                "docUrl": pdf_url1,
                "docSAS": pdf_sas1
            }
        },  
        {
            "recordId": "record2",
            "data": { 
                "docUrl": pdf_url1,
                "docSAS": pdf_sas1
            }
        },
    ]
}

response = requests.post(url, json=data)
print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4))
except requests.exceptions.JSONDecodeError:
    print(response.text)

In [ ]:
# Extract the page number from the markdown string
markdown = """     Pageer 43

page 456
    
    Monthly Savings
    | Month    | Savings |Details      |
    | -------- | ------- |------------ |
    | January  | $250    | for holiday |
    | February | $80     | pension     |
    | March    | $420    | new cat     |
    
    Savings were significantly lower in February. This is surprising because it is a short month and contributing to pension shuld be a priority.
    """
    
page_number = markdown.split('\n')[0].replace("Page ", "").strip()
print(page_number)

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobSasPermissions, generate_blob_sas, BlobClient

pdf_url=pdf_url1+"?"+pdf_sas1
print(pdf_url)
blob_client = BlobClient.from_blob_url(pdf_url)
pdf_bytes = blob_client.download_blob().readall()

In [ ]:
import datetime

blob_connection_string = os.environ["BLOB_STORAGE_ACCOUNT_CONNECTION_STRING"]
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client("customskillimages")


def get_account_key_from_connection_string(connection_string):
    parts = connection_string.split(';')
    for part in parts:
        if part.startswith('AccountKey='):
            return part[len('AccountKey='):]
    return None



account_key = get_account_key_from_connection_string(blob_connection_string)
print(account_key)
sas_token = generate_blob_sas(
    account_key=account_key,
    account_name=blob_service_client.account_name,
    container_name=container_client.container_name,
    blob_name=blob_client.blob_name,
    permission=BlobSasPermissions(read=True),
    expiry=datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=1)
)